In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/tarasbohdan/Desktop/university/Experiments-AAAI-24


In [4]:
import copy

from notebooks.mult_repair_levels_student_performance.StudentPerformanceDataset import StudentPerformanceDataset
from source.preprocessing import get_simple_preprocessor
from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

from source.preprocessing import lfr

from virny.preprocessing.basic_preprocessing import preprocess_dataset

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [5]:
class Config:
  file_name = './data/student_performance/student-mat.csv'

In [6]:
data_loader = StudentPerformanceDataset(Config.file_name)
data_loader.X_data.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,Mjob,Fjob,reason,guardian,sex
0,18,4,4,2,2,0,4,3,4,1,1,3,6,0,0,at_home,teacher,course,mother,F
1,17,1,1,1,2,0,5,3,3,1,1,3,4,0,0,at_home,other,course,father,F
2,15,1,1,1,2,3,4,3,2,2,3,3,10,0,0,at_home,other,other,mother,F
3,15,4,2,1,3,0,3,2,2,1,1,5,2,1,1,health,services,home,mother,F
4,16,3,3,1,2,0,4,3,2,1,2,5,4,0,1,other,other,home,father,F


In [7]:
init_data_loader = copy.deepcopy(data_loader)

In [8]:
exp_iter_num = 7
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
test_set_fraction = TEST_SET_FRACTION

In [9]:
data_loader.categorical_columns = [col for col in data_loader.categorical_columns if
                                           col not in ('sex')]
data_loader.X_data['sex_binary'] = data_loader.X_data['sex'].apply(lambda x: 1 if x == 'M' else 0)
data_loader.full_df = data_loader.full_df.drop(['sex'], axis=1)
data_loader.X_data = data_loader.X_data.drop(['sex'], axis=1)

# Preprocess the dataset using the defined preprocessor
column_transformer = get_simple_preprocessor(data_loader)
base_flow_dataset = preprocess_dataset(data_loader, column_transformer, test_set_fraction, experiment_seed)
base_flow_dataset.init_features_df = init_data_loader.full_df.drop(init_data_loader.target, axis=1,
                                                                   errors='ignore')
base_flow_dataset.X_train_val['sex_binary'] = data_loader.X_data.loc[
    base_flow_dataset.X_train_val.index, 'sex_binary']
base_flow_dataset.X_test['sex_binary'] = data_loader.X_data.loc[base_flow_dataset.X_test.index, 'sex_binary']

In [10]:
base_flow_dataset.X_train_val.head()

,cat__Mjob_at_home,cat__Mjob_health,cat__Mjob_other,cat__Mjob_services,cat__Mjob_teacher,cat__Fjob_at_home,cat__Fjob_health,cat__Fjob_other,cat__Fjob_services,cat__Fjob_teacher,...,num__famrel,num__freetime,num__goout,num__Dalc,num__Walc,num__health,num__absences,num__G1,num__G2,sex_binary
324,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.994710,-0.249211,-0.087835,0.580313,0.515293,-1.115062,-0.688980,0.765641,0.777593,0
184,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-3.156443,-1.246056,-0.985706,-0.547619,-0.266141,-1.827463,0.950317,0.765641,0.777593,0
79,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-2.075577,0.747634,-0.087835,-0.547619,-0.266141,-0.402661,0.716132,-1.306094,-1.286019,0
346,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.167022,-0.249211,-0.985706,-0.547619,-0.266141,0.309740,0.364854,0.765641,0.777593,1
299,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-3.156443,0.747634,-0.985706,0.580313,-0.266141,-1.827463,-0.103517,0.765641,0.777593,1


In [11]:
cur_base_flow_dataset = lfr(base_flow_dataset, True)

Mean difference before:  -0.12
Disparate impact before: 0.84
Mean difference after:  0.00
Disparate impact after: 1.00


In [12]:
cur_base_flow_dataset.X_train_val.head()

,cat__Mjob_at_home,cat__Mjob_health,cat__Mjob_other,cat__Mjob_services,cat__Mjob_teacher,cat__Fjob_at_home,cat__Fjob_health,cat__Fjob_other,cat__Fjob_services,cat__Fjob_teacher,...,num__failures,num__famrel,num__freetime,num__goout,num__Dalc,num__Walc,num__health,num__absences,num__G1,num__G2
324,0.590410,0.379161,0.546567,0.527648,0.393789,0.491360,0.366302,0.599981,0.303586,0.367536,...,0.414159,0.669467,0.550259,0.412498,0.191511,0.481917,0.643229,0.566015,0.501933,0.656017
184,0.547686,0.358401,0.592104,0.547107,0.445755,0.504554,0.373265,0.565404,0.373132,0.436816,...,0.410513,0.629042,0.553452,0.392025,0.208947,0.488573,0.649924,0.577743,0.504278,0.696812
79,0.596685,0.408034,0.580705,0.502078,0.382680,0.458566,0.363669,0.609214,0.318601,0.377188,...,0.401599,0.654021,0.563513,0.401036,0.211167,0.496999,0.639221,0.541187,0.478972,0.656626
346,0.510903,0.337451,0.566718,0.487882,0.473057,0.453147,0.352937,0.477255,0.416405,0.461516,...,0.473101,0.635133,0.559625,0.406253,0.248846,0.446440,0.618789,0.579802,0.535092,0.695577
299,0.509184,0.349592,0.607299,0.499488,0.476223,0.457360,0.358074,0.496064,0.433796,0.481361,...,0.446992,0.612309,0.566842,0.387906,0.251723,0.469710,0.628883,0.571753,0.514414,0.710742
